In [1]:
import os
from pathlib import Path

import pandas as pd

TABLE_STORE_PATH = "../../../schema/fixtures/workflow_inputs"

tables: dict[str, pd.DataFrame] = {}
for file in os.listdir(TABLE_STORE_PATH):
    if file.endswith(".csv"):
        path = Path(TABLE_STORE_PATH) / file
        table = pd.read_csv(
            path,
        )
        tables[file.removesuffix(".csv")] = table

In [2]:
import json

with Path(
    "../../../schema/fixtures/workflow/verbs/join/cross/workflow.json"
).open() as workflow_file:
    workflow = json.load(workflow_file)

In [3]:
from datashaper import Workflow
from datashaper.table_store.disk_cache_table_store import DiskCacheTableStore

with DiskCacheTableStore() as table_store:
    graph = Workflow(
        schema=workflow,
        input_tables=tables,
        schema_path="../../../schema/workflow.json",
        table_store=table_store,
    )
    await graph.run()
    result = graph.output("expected")

result

,x,y,u,v
0,A,1,A,4
1,A,1,B,5
2,A,1,D,6
3,B,2,A,4
4,B,2,B,5
5,B,2,D,6
6,C,3,A,4
7,C,3,B,5
8,C,3,D,6
